In [1]:
import numpy as np
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import meteor_score
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer 
import re

In [2]:
file_path_1 = "../datasets/HINGE dataset/test.csv"
file_path_2 = "../datasets/HINGE dataset/valid.csv"
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

In [3]:
df1

,English,Hindi,Hinglish,Average rating,Disagreement
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...,7,4
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...,8,1
2,Compete with one another to achieve forgivenes...,तुम अपने परवरदिगार के (सबब) बख़्शिस की और बेहि...,tum apne paravardigar ke sabab bashis ki aur b...,3,5
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...,8,1
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...,9,2
...,...,...,...,...,...
786,For God is sufficient as witness between us an...,"""हमारे और तुम्हारे बीच अल्लाह ही एक गवाह काफ़ी...",hmare aur tumhare bich allah hi ek sufficient ...,6,1
787,But the fact is half the colliery is reeling u...,सच पूछो तो आधी कोलियरी उसकी कर्जदार है।\n,fact poochho to aadhi colliery uski karjdar ha...,4,4
788,"A former Mujahideen commander , Haq 's ties wi...",क 80 के दशक के सोवियत-अफगान युद्ध के समय से ही...,k 80 ke dashak ke ties yuddh ke samay se hi si...,4,4
789,"In fact, Aperture time is also an exposure tim...",वास्तव में एपर्चर काल ऐसी किरणों को दिया गया व...,fact men aperture time aesi exposure ko diya g...,6,1


In [4]:
df = pd.concat([df1, df2], ignore_index = True)
df

,English,Hindi,Hinglish,Average rating,Disagreement
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...,7,4
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...,8,1
2,Compete with one another to achieve forgivenes...,तुम अपने परवरदिगार के (सबब) बख़्शिस की और बेहि...,tum apne paravardigar ke sabab bashis ki aur b...,3,5
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...,8,1
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...,9,2
...,...,...,...,...,...
1181,that God may distinguish the corrupt from the ...,ताकि अल्लाह नापाक को पाक से छाँटकर अलग करे और ...,taki allah napak ko pak se chhantakar alag kar...,4,7
1182,No such interface ‘% s'\n,कोई अंतरफलक ‘% s 'नहीं\n,koee interface s nhin\n,8,0
1183,Saccus endolymphaticus is also known as endoly...,अन्तरुदक कोश को अन्तः कर्णोदक कोश के रूप में भ...,antrudak sac ko anta endolymphatic sac ke roop...,7,1
1184,Retrieving quota information for folder '% s'\n,% s फ़ोल्डर के लिए के लिए कोटा प्राप्त कर रहा ...,s folder ke lie ke lie quota information kar r...,4,5


In [5]:
df = df[(df['Average rating'] >= 5) & (df['Disagreement'] <= 2)]
df

,English,Hindi,Hinglish,Average rating,Disagreement
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...,8,1
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...,8,1
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...,9,2
5,"Deva Devi, the lovely maiden, was the pride of...",सुंदर कन्या देवदेवी ब्रह्मा की सृष्टि की गौरवप...,lovely maiden deva brahma ki creation ki pride...,8,0
6,Registration under Shops and Establishment Act\n,दुकानें और प्रतिष्ठान अधिनियम के तहत पंजीकरण\n,shops aur establishment act ke tahat registrat...,10,0
...,...,...,...,...,...
1178,"Ripe plantains, some sweets and other ingredie...","जिस डोंगे में चावल भरा होता है, उसमें पके केले...","jis donge filled with rice hota hai, usmen pak...",8,0
1180,"I shall not , however , attempt to infer that ...","बहरहाल , मैं यह अनुमान नहीं लगाना चाहता था कि ...","baharhal , main yah inference nahin lagana cha...",8,1
1182,No such interface ‘% s'\n,कोई अंतरफलक ‘% s 'नहीं\n,koee interface s nhin\n,8,0
1183,Saccus endolymphaticus is also known as endoly...,अन्तरुदक कोश को अन्तः कर्णोदक कोश के रूप में भ...,antrudak sac ko anta endolymphatic sac ke roop...,7,1


In [6]:
native_hindi_sentences = df['Hindi'].values
english_sentences = df['English'].values

In [7]:
native_hindi_sentences

array(['यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण सिक्किम का प्रत्यक्ष अनुभव होता है और रोमांचक खेलों की असीमित संभावनाएं उपलब्ध हैं।\n',
       'विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिधि और जनहित संरक्षक के रूप में उन्हें विधान, धन और वित्त संबंधी मामलों में अत्यंत सतर्क रहना होगा।\n',
       'कैप को इस तरह बनाया गया है कि यह गर्भाशय के मुख को इस तरह बंद कर देता है कि हवा भी भीतर नहीं जा सकती।\n',
       'सुंदर कन्या देवदेवी ब्रह्मा की सृष्टि की गौरवपूर्ण रचना थी।\n',
       'दुकानें और प्रतिष्ठान अधिनियम के तहत पंजीकरण\n',
       'पर शारीरिक प्रेम दूसरे तट-आत्म-हनन-पर पहुँचने का सेतु मात्र है।\n',
       'उसे ऐसा लग रहा था मानो चारों दिशाएं उसे निविड़ता से घेर रही हों, आकाश मानो उसे छू रहा हो।\n',
       'शीग्र ही मानसिंह अंग्रेजों के साथ सौदे बाजी करने तात्या टोपे लगा।\n',
       'भाषा मेरा गणित नहीं समेट सकती.\n',
       'आज का यह पवित्र दिन हम उनकी स्मृति में समर्पित करते हैं।\n',
       'तुम्हे दुख में देखकर मुझे भी दुख हो रहा है, इसलिए तुम लोगों को बचाने की मैं जी-जान

In [8]:
# Native hindi to english translations using the Facebook model
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

facebook_bleu_scores = []
facebook_gleu_scores = []
facebook_meteor_scores = []

# translate Hindi to English
tokenizer.src_lang = "hi"
for i, hindi_text in enumerate(native_hindi_sentences):
    # print(i)
    encoded_hi = tokenizer(hindi_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    # print(translated_text)
    # print(translated_text[0].split())
    reference = [english_sentences[i].split()]
    transformed_tokens = translated_text[0].split()

    bleu_score = sentence_bleu(reference, transformed_tokens)
    gleu_score = sentence_gleu(reference, transformed_tokens)
    met_score = meteor_score(reference, transformed_tokens)

    facebook_bleu_scores.append(bleu_score)
    facebook_gleu_scores.append(gleu_score)
    facebook_meteor_scores.append(met_score)

    if i % 10 == 0:
        print("Translating sentence ", i+1)
        print('Original Hindi text :', hindi_text)
        print('Original English Conversion :', english_sentences[i])
        print('Translated to English :', translated_text[0])
        print('Average BLEU score :', np.mean(facebook_bleu_scores), '\n')
        print('Average GLEU score :', np.mean(facebook_gleu_scores), '\n')
        print('Average METEOR score :', np.mean(facebook_meteor_scores), '\n')
        print()

Translating sentence  1
Original Hindi text : यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण सिक्किम का प्रत्यक्ष अनुभव होता है और रोमांचक खेलों की असीमित संभावनाएं उपलब्ध हैं।

Original English Conversion : From popular home stays where one can get a firsthand experience of rural Sikkim to endless possibilities for adventure sports.

Translated to English : Here is a popular domestic shelter where you have a direct experience of rural Sikkim and unlimited possibilities of exciting games are available.
Average BLEU score : 0.12259149958656422 

Average GLEU score : 0.16279069767441862 

Average METEOR score : 0.33905005169372987 




C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Nithish\anaconda3\envs\mt_final_project\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

Translating sentence  11
Original Hindi text : तुम्हे दुख में देखकर मुझे भी दुख हो रहा है, इसलिए तुम लोगों को बचाने की मैं जी-जान से कोशिश करूंगा।

Original English Conversion : I 'll try my best and see if I can be of any help.

Translated to English : I’m also sad when I see you in sadness, so I’ll try to save you with G-Jan.
Average BLEU score : 0.029744701842662 

Average GLEU score : 0.1226136678983496 

Average METEOR score : 0.3471363948733488 


Translating sentence  21
Original Hindi text : फीताकृमि ग्रसन में मनुष्य प्राथमिक पोषी या आश्रयी है

Original English Conversion : In tapeworm infestation human are the primary host.

Translated to English : Man is a primary food or a shelter.
Average BLEU score : 0.04996897583012464 

Average GLEU score : 0.15548839056234154 

Average METEOR score : 0.37717598601461283 


Translating sentence  31
Original Hindi text : कॉरपोरेट सामाजिक उत्तरदायित्व का विचार भारत के लिए नया नहीं है।

Original English Conversion : The notion of CSR is not

In [9]:
average_bleu_score = np.mean(facebook_bleu_scores)
average_gleu_score = np.mean(facebook_gleu_scores)
average_meteor_score = np.mean(facebook_meteor_scores)
print('Average BLEU Score for Test Set using Facebooks model: ', average_bleu_score)
print('Average GLEU Score for Test Set using Facebooks model: ', average_gleu_score)
print('Average METEOR Score for Test Set using Facebooks model: ', average_meteor_score)

Average BLEU Score for Test Set using Facebooks model:  0.09047040143142776
Average GLEU Score for Test Set using Facebooks model:  0.2029706548654746
Average METEOR Score for Test Set using Facebooks model:  0.4158111326524848


In [10]:
# hinglish_sentences = df['Hinglish']
# print(hinglish_sentences)

# hinglish_sentences = hinglish_sentences.values
# print(type(hinglish_sentences))
# print(hinglish_sentences.shape)

In [11]:
# # Testing out Google Transliterate package
# from google.transliteration import transliterate_word
# def transliterate_all_alphanumeric_parts(sentence, lang_code='hi'):
#     transliterated_sentence = []
#     words = sentence.split()

#     for word in words:
#         # Use regex to find all alphanumeric parts in the word
#         parts = re.findall(r'[a-zA-Z0-9]+|[^a-zA-Z0-9]+', word)
        
#         transliterated_word = ""
        
#         for part in parts:
#             if part.isalnum():  # Transliterate only alphanumeric parts
#                 try:
#                     # Transliterate the alphanumeric part
#                     suggestions = transliterate_word(part, lang_code=lang_code)
#                     transliterated_part = suggestions[0] if suggestions else part
#                 except IndexError:
#                     # If transliteration fails, keep the alphanumeric part as is
#                     transliterated_part = part
#             else:
#                 # Keep non-alphanumeric parts (like "-" or spaces) as is
#                 transliterated_part = part
            
#             transliterated_word += transliterated_part
        
#         transliterated_sentence.append(transliterated_word)

#     return ' '.join(transliterated_sentence)

# google_bleu_scores = []
# google_gleu_scores = []
# google_meteor_scores = []
# lang = 'hi'
# for i in range(hinglish_sentences.size):

#     # print(i)
#     # transformed_sentence = transliterate_text(hinglish_sentences[i],lang_code = lang)
#     transformed_sentence = transliterate_all_alphanumeric_parts(hinglish_sentences[i], lang_code=lang)
    
#     reference = [hindi_true[i].split()]
#     transformed_tokens = transformed_sentence.split()
    
#     bleu_score = sentence_bleu(reference, transformed_tokens)
#     gleu_score = sentence_gleu(reference, transformed_tokens)
#     met_score = meteor_score(reference, transformed_tokens)
#     google_bleu_scores.append(bleu_score)
#     google_gleu_scores.append(gleu_score)
#     google_meteor_scores.append(met_score)

#     if i % 10 == 0:
#         print("Transliterating sentence ", i+1)
#         print('Original Code mized text :', hinglish_sentences[i])
#         print('Original Native Hindi convertion :', hindi_true[i])
#         print('Transliterated to Hindi :', transformed_sentence)
#         print('Average BLEU score :', np.mean(google_bleu_scores), '\n')
#         print('Average GLEU score :', np.mean(google_gleu_scores), '\n')
#         print('Average METEOR score :', np.mean(google_meteor_scores), '\n')
#         print()

In [12]:
# average_bleu_score = np.mean(google_bleu_scores)
# average_gleu_score = np.mean(google_gleu_scores)
# average_meteor_score = np.mean(google_meteor_scores)
# print('Average BLEU Score for Dev Set using Google transliterator: ', average_bleu_score)
# print('Average GLEU Score for Dev Set using Google transliterator: ', average_gleu_score)
# print('Average METEOR Score for Dev Set using Google transliterator: ', average_meteor_score)
# print('Google Transliterate model is very accurate but takes time to transliterate the sentence')